In [1]:
#%%
import sys
import pandas as pd
import numpy as np
import os
import calendar
import psycopg2
from pathlib import Path
from loguru import logger
from dotenv import load_dotenv
from datetime import datetime, timedelta
from unidecode import unidecode
from avay_bq import AvayBQServiceAcc

from slack import WebClient
from slack.errors import SlackApiError

import calendar
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from math import floor, ceil

from da_utils.client.gmail_client import GmailClient
from da_utils.repository.gmail.gmail_repository import GmailRepository
from da_utils.repository.gmail.custom_types import GmailMessage

from da_utils.client.google_sheets_client import GoogleSheetsClient
from da_utils.repository.google_sheets.google_sheets_repository import GoogleSheetsService

from da_utils.client.drive_client import GoogleDriveClient
from da_utils.repository.drive.drive_repository import GoogleDriveService

from da_utils.client.slack_client import SlackClient
from da_utils.repository.slack.slack_repository import SlackRepository

from da_utils.client.bigquery_client import BigqueryClient
from da_utils.repository.bigquery.bigquery_repository import BigqueryService

from da_utils.conver_phone_number.phone_number import convert_to_new_prefix

load_dotenv()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 2000

In [2]:
gmail_client = GmailClient(Path(os.environ.get("GOOGLE_CREDENTIALS_PATH")), Path(os.environ.get("GMAIL_TOKEN_PATH")))
gmail_repo = GmailRepository(gmail_client)

drive_client = GoogleDriveClient( )
drive_repo = GoogleDriveService(drive_client)

ggsheet_client = GoogleSheetsClient( )
ggsheet_repo = GoogleSheetsService(ggsheet_client)

slack_client = SlackClient(token=os.environ.get("SLACK_BOT_AUTHEN"))
slack_repo = SlackRepository(client=slack_client)

conn = psycopg2.connect(
    host = "v1proxy-new-postgres-replica.cvhqeqp3dmlv.ap-southeast-1.rds.amazonaws.com",
    dbname = "score_api_production",
    user = "analytic",
    password = "COng@tr0ngcuctacuctac",
    port = "5432"
)

bot_auth_token = os.environ.get("SLACK_BOT_AUTHEN")
user_auth_token = os.environ.get("SLACK_USER_AUTHEN")
clientSlack = WebClient(token = bot_auth_token)

channel_id = "C04MYU3L7LN"
user_id = "U0480LX5468"
linh_id = "U047QMYB9TQ"


In [3]:
def dayrange_recon(day_from_curr: int = 40) -> tuple:
    curr_time = datetime.now()
    recon_time =(curr_time - timedelta(day_from_curr))
    sys_time = curr_time - timedelta(days=180)
    last_month_time = (recon_time - timedelta(30))
    last_2month_time = (recon_time - timedelta(60))
    _, end_date = calendar.monthrange(int(recon_time.strftime("%Y")), int(recon_time.strftime("%m")))

    recon_month = recon_time.strftime("%m-%Y")
    last_month = last_month_time.strftime("%Y-%m")
    last_2month = last_2month_time.strftime("%Y-%m")
    month = recon_time.strftime("%Y-%m")
    start_date = recon_time.strftime("%Y-%m") + "-" + "01"
    end_date = recon_time.strftime("%Y-%m") + "-" + str(end_date)
    sys_date = sys_time.strftime("%Y-%m-%d")
    return recon_month, month, recon_time, start_date, end_date, last_month, last_2month, sys_date

recon_month, month, recon_time, start_date, end_date, last_month, last_2month, sys_date = dayrange_recon()


In [4]:
def create_or_get_folder(recon_month, parent_folder_id):
    folder_check = drive_repo.list_folder_names(parent_folder_id =  parent_folder_id)
    if recon_month not in folder_check:
        folder_id = drive_repo.create_folder(folder_name= recon_month, parent_folder_id =  parent_folder_id)
    else:
        folder_id = drive_repo.get_folders(folder_name= recon_month, parent_folder_id =  parent_folder_id)
        folder_id = folder_id[0]
    return folder_id

def create_or_get_workbook(parent_folder_id, wb_name):
    wb_check = drive_repo.list_file_names(parent_folder_id= parent_folder_id)
    if wb_name not in wb_check:
        wb = ggsheet_repo.create_spreadsheet(spreadsheet_name = wb_name,parent_folder_id = parent_folder_id)
    else:
        spreadsheet_key = drive_repo.get_files(file_name = wb_name,parent_folder_id = parent_folder_id)
        spreadsheet_key = spreadsheet_key[0]
        wb = ggsheet_repo.open_spreadsheet(spreadsheet_key = spreadsheet_key)
    return wb

def create_or_update_sheet(wb, new_sheet, input_df, starting_cell: str = 'A1'):
    sheet_names = [worksheet.title for worksheet in wb.worksheets()]
    if new_sheet not in sheet_names:
        wb.add_worksheet(title=new_sheet, rows=100, cols=20)
        ws = wb.worksheet(new_sheet)
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell=starting_cell)
    else:
        ws = wb.worksheet(new_sheet)
        ws.clear()
        ggsheet_repo.write_df_to_sheet(worksheet = ws, df_to_write = input_df, starting_cell=starting_cell)
    if 'Sheet1' in sheet_names: 
        wb.del_worksheet(wb.worksheet('Sheet1')) 
    else:
        pass
    return

parent_folder_id = '1qL1L1X_Tqzcj9rJr6wjRLXJC21tC23QG'


In [5]:
folder_month = create_or_get_folder(recon_month = month, parent_folder_id = parent_folder_id)
path = '/home/linhnguyen/04.Reconcile/04.VPBank/'

In [6]:
summary_fee_report =create_or_get_workbook(parent_folder_id = folder_month, wb_name = f'summary_fee_{month}')
vpb_converted_report = create_or_get_workbook(parent_folder_id = folder_month, wb_name = f'vpb_converted_{month}')
vpb_converted_cycle_report = create_or_get_workbook(parent_folder_id = folder_month, wb_name = f'vpb_converted_{month}_recon_cycle')
vpb_statement_report = create_or_get_workbook(parent_folder_id = folder_month, wb_name = f'vpb_statement_{month}')

I - GET FILE FROM GMAIL

In [7]:
CLIENT_EMAIL="maidtt12@vpbank.com.vn"
QUERY_STRING=f'[TS-VPBANK-VIETTEL] {month} from:{CLIENT_EMAIL} has:attachment'

gmail_messages = gmail_repo.search_messages_by_query(query=QUERY_STRING)
for message in gmail_messages:
    attachment_pair_id = gmail_repo.query_attachments_from_email_message(message=message, attachment_ex_filter="xlsx")
    for attachment_pair_id in attachment_pair_id:
        if attachment_pair_id is not None:
            print(attachment_pair_id)
            upl = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id, 
                                                            password = 'hant135@02', sheet_name = 'Inhouse_Loan')
            cc = gmail_repo.read_excel_attachment_to_pandas(attachment_pair = attachment_pair_id, 
                                                            password = 'hant135@02', sheet_name = 'Inhouse_Card')
        else: pass

upl.columns = upl.columns.str.lower()
cc.columns = cc.columns.str.lower()
upl.head()

messageId='18a69d654b5cfd47' attachmentId='ANGjdJ_JWQDFfOzXDY-vmg1aHBTTuWWrTrORQhofzTnbOCQuQe5gJSPGnGfxJXyHv01A1wqH7ttQZNxC7_m1Tbu5viIuOL4BdWyeV_gRR12HxgrJEKL3haVYkkRRIfbOltPRv5Ru6Va2ZGrmC_D6BYRpVX7onTvjKG1uChr2COVXa3BedGMV-FsVZS6lxQn4akgrwc2r1HtvtZeC4OGUlzgxs_UZkXf3itr4TRVnzVxu_552Ff_aEPpg-bqbBzmEembbAYCdMkc6FThRtdjIYL2m5Tm6Rhzan-KrCPRoklyeTassKz7ViveSpxTYlJvEC9xf75QMwIiGYCSPdB4p1dHq86ov9f4aXLKsQxzboV8EL43KCAs0zYnO7d2NQZLnj3EFYsiTlhehwLJtxJQJ' attachment_file_name='TRUSTING_SOCIAL_20230802.xlsx'


,app,branch_id,acctno,cif,customer_name,national_id,dist_date,disb_amt,product_name,product_detail,app_id_c,campaign_code,bi_project,score_band,phone_number,province_city,introducer_fee
0,LOAN NEW,VN0010005,LD2318401280,12241015,LE DAC NAM,027081003743,2023-07-03,80000000,10. UPL,10. UPL,LN2306309602532,TRUSTSOCIAL,Outsource,B05,0989071888,BAC NINH,2000000
1,LOAN NEW,VN0010005,LD2318401400,12213899,DOAN DINH MANH,042084001333,2023-07-03,30000000,10. UPL,10. UPL,LN2306309599720,TRUSTSOCIAL,Outsource,B05,0962323836,HA TINH,1800000
2,LOAN NEW,VN0010005,LD2318200269,12215627,VU TRI DUNG,024094007028,2023-07-01,80000000,10. UPL,10. UPL,LN2306309597575,TRUSTSOCIAL,Outsource,B05,0988564186,BAC GIANG,2000000
3,LOAN NEW,VN0010005,LD2318200222,12212999,NGUYEN VAN TO,051081011479,2023-07-01,30000000,10. UPL,10. UPL,LN2306309598639,TRUSTSOCIAL,Outsource,B05,0979498001,QUANG NGAI,1800000
4,LOAN NEW,VN0010005,LD2318701811,2805394,NGO THI NGOC BICH,075186017213,2023-07-06,75000000,10. UPL,10. UPL,LN2307049627757,TRUSTSOCIAL,Outsource,B05,0966371101,Dong Nai,2000000


In [8]:
def upl_cleaned(upl_hq):
    # Add new columns and select subset of columns
    upl_hq['introducer_fee'] = upl_hq['introducer_fee'].astype(float)
    df_upl = (upl_hq.assign(month=upl_hq['dist_date'].str.slice(stop=7),
                            receiver='HQ',
                            product_type='UPL',
                            fee_no_vat=upl_hq['introducer_fee']/1.1)
                   .loc[:, ['app_id_c', 'product_type', 'product_detail', 'receiver', 'phone_number', 'customer_name',
                              'score_band', 'dist_date', 'disb_amt', 'fee_no_vat', 'introducer_fee', 'national_id']]
             )
    
    # Rename columns and convert product_detail to string
    df_upl = (df_upl.rename(columns={'dist_date': 'disbursed_date', 
                                     'disb_amt': 'loan_amount', 
                                     'introducer_fee': 'fee_w_vat', 
                                     'app_id_c': 'contract_no'})
                     .astype({'product_detail': 'str',
                                'phone_number': 'str'})
             )
    
    # Check for duplicates
    if df_upl['contract_no'].duplicated().any():
        print('Warning: duplicates found in contract_no column')
    
    # Convert disbursed_date to date format
    df_upl['disbursed_date'] = pd.to_datetime(df_upl['disbursed_date'])
    
    return df_upl
cleaned_upl_df = upl_cleaned(upl)
cleaned_upl_df.head()

,contract_no,product_type,product_detail,receiver,phone_number,customer_name,score_band,disbursed_date,loan_amount,fee_no_vat,fee_w_vat,national_id
0,LN2306309602532,UPL,10. UPL,HQ,0989071888,LE DAC NAM,B05,2023-07-03,80000000,1.818182e+06,2000000.0,027081003743
1,LN2306309599720,UPL,10. UPL,HQ,0962323836,DOAN DINH MANH,B05,2023-07-03,30000000,1.636364e+06,1800000.0,042084001333
2,LN2306309597575,UPL,10. UPL,HQ,0988564186,VU TRI DUNG,B05,2023-07-01,80000000,1.818182e+06,2000000.0,024094007028
3,LN2306309598639,UPL,10. UPL,HQ,0979498001,NGUYEN VAN TO,B05,2023-07-01,30000000,1.636364e+06,1800000.0,051081011479
4,LN2307049627757,UPL,10. UPL,HQ,0966371101,NGO THI NGOC BICH,B05,2023-07-06,75000000,1.818182e+06,2000000.0,075186017213


In [9]:
def cc_cleaned(cc_hq):
    # Add new columns and select subset of columns
    cc_hq['introducer_fee'] = cc_hq['introducer_fee'].astype(float)
    df_cc = (cc_hq.assign(introducer_fee=495000,
                           month=cc_hq['dist_date'].str.slice(stop=7),
                           receiver='HQ',
                           product_type='CC',
                           fee_no_vat=(495000/1.1),
                           disb_amt=cc_hq['card_limit'] * -1)
                   .loc[:, ['app_id_c', 'product_type', 'product_detail', 'receiver', 'phone_number', 'customer_name',
                            'score_band', 'dist_date', 'disb_amt', 'fee_no_vat', 'introducer_fee', 'national_id']]
             )

    # Rename columns and convert product_detail to string
    df_cc = (df_cc.rename(columns={'dist_date': 'disbursed_date', 
                                    'disb_amt': 'loan_amount', 
                                    'introducer_fee': 'fee_w_vat', 
                                    'app_id_c': 'contract_no'})
                    .astype({'product_detail': 'str',
                            'phone_number': 'str'})
             )

    # Check for duplicates
    if df_cc['contract_no'].duplicated().any():
        print('Warning: duplicates found in contract_no column')

    # Convert disbursed_date to date format
    df_cc['disbursed_date'] = pd.to_datetime(df_cc['disbursed_date'])
    
    return df_cc
cleaned_cc_df = cc_cleaned(cc)
cleaned_cc_df.head()

,contract_no,product_type,product_detail,receiver,phone_number,customer_name,score_band,disbursed_date,loan_amount,fee_no_vat,fee_w_vat,national_id


In [10]:
cleaned_cc_df.to_csv('df_cc_rdata.csv', index = False)

In [11]:
df_all = pd.concat([cleaned_upl_df, cleaned_cc_df], axis = 0)
df_all['phone_number'] = convert_to_new_prefix(df_all['phone_number'])
df_all.head()

,contract_no,product_type,product_detail,receiver,phone_number,customer_name,score_band,disbursed_date,loan_amount,fee_no_vat,fee_w_vat,national_id
0,LN2306309602532,UPL,10. UPL,HQ,84989071888,LE DAC NAM,B05,2023-07-03,80000000,1.818182e+06,2000000.0,027081003743
1,LN2306309599720,UPL,10. UPL,HQ,84962323836,DOAN DINH MANH,B05,2023-07-03,30000000,1.636364e+06,1800000.0,042084001333
2,LN2306309597575,UPL,10. UPL,HQ,84988564186,VU TRI DUNG,B05,2023-07-01,80000000,1.818182e+06,2000000.0,024094007028
3,LN2306309598639,UPL,10. UPL,HQ,84979498001,NGUYEN VAN TO,B05,2023-07-01,30000000,1.636364e+06,1800000.0,051081011479
4,LN2307049627757,UPL,10. UPL,HQ,84966371101,NGO THI NGOC BICH,B05,2023-07-06,75000000,1.818182e+06,2000000.0,075186017213


In [12]:
cross_sell = df_all[df_all.duplicated(['phone_number', 'national_id'], keep=False)].sort_values('phone_number')
cross_sell

,contract_no,product_type,product_detail,receiver,phone_number,customer_name,score_band,disbursed_date,loan_amount,fee_no_vat,fee_w_vat,national_id


In [13]:
sum1 = df_all.groupby(['phone_number', 'national_id']).agg(
    count=('phone_number', 'count'),
    la=('loan_amount', 'sum'),
    fee_no_vat=('fee_no_vat', 'sum')
).reset_index()

sum_count_gt_1 = sum(sum1[sum1['count'] > 1]['count']) / 2
total_sum_count = sum(sum1['count'])
final_result = (sum_count_gt_1 / total_sum_count) * 100

# Print the results
print(f"Total xsell leads:", sum_count_gt_1)
print("Total disbursed leads:", total_sum_count)
print("Xsell rate:", final_result)


Total xsell leads: 0.0
Total disbursed leads: 31
Xsell rate: 0.0


2. QUERY DATA TS

In [14]:
vp_query = f"""select id as lead_id, phone_number,(other ->> 'id_card_number') as national_id,bank_code, 
                 bound_code, to_char(sent_at AT TIME ZONE 'UTC','YYYY-MM-dd') as sent_at,
                 date(created_at at time zone 'UTC') as created_at,
                 (case when date(created_at at time zone 'UTC') < '2020-01-10' then 'V2' 
                        when date(created_at at time zone 'UTC') between '2020-01-10' and '2022-05-11' then 'V3'
                        when date(created_at at time zone 'UTC') >= '2022-05-12' then 'V4'
                        else 'NA' end) as cs_version,
                 other->>'sender' as sender,
                 other->>'province' as location,
                 score,
                 score_range,
                 telco_code,
                 (case when (pre_scoring_data ->> 'is_qualified')::boolean = true then 1 else 0 end) as qualified,
                 (case when was_sent=TRUE AND send_info IS NOT NULL then 1 else 0 end) as fwd,
                 (case when (other ->> 'recycled')::boolean = true then 1 else 0 end) as recycled
                 
                 from phone_infos 
                 
                 where bank_code like 'vp%' 
                 and was_sent = 't' 
                 and date(sent_at AT TIME ZONE 'UTC') > '2021-12-01'
                 and date(sent_at AT TIME ZONE 'UTC') <= date('",{end_date},"')
                 order by date(sent_at AT TIME ZONE 'UTC')  DESC"""
ts_sent = pd.read_sql(vp_query, conn)
ts_sent

/tmp/ipykernel_3636213/2895507222.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ts_sent = pd.read_sql(vp_query, conn)


In [13]:
ts_sent['score'] = ts_sent.apply(lambda x: x['score'] if pd.notna(x['score']) else x['score_range'][0:3], axis=1)
ts_sent = ts_sent.astype({'lead_id': int, 'score': int})

filtered_sent = ts_sent.query('qualified == 1 & fwd == 1')
max_sent_at = filtered_sent.groupby('phone_number')['sent_at'].max()
latest_sent = pd.merge(filtered_sent, max_sent_at, on=['phone_number', 'sent_at'])
latest_sent.sort_values(by='sent_at', ascending=False, inplace=True)

In [14]:
latest_sent['receiver'] = np.select(
    [
        (latest_sent['bank_code'] == 'vpbank_hq'),
        (latest_sent['bank_code'] == 'vpbank') & (latest_sent['bound_code'].isin(['CC_LADY', 'CC_UP', 'CC_VNA'])),
        (latest_sent['bank_code'] == 'vpbank') & (latest_sent['bound_code'].isin(['HQ_B3', 'HQ_B4', 'HQ_B5']))
    ],
    [
        'HQ',
        'HQ',
        'HQ'
    ],
    default='TP'
)

In [15]:
latest_sent['bound_code'] = latest_sent['bound_code'].fillna('unknown').astype(str)

bound_code_split = latest_sent['bound_code'].str.split('_', expand=True)
product_type = np.where(bound_code_split[0] == 'CC', 'CC', 'UPL')

latest_sent['product_type'] = product_type

In [16]:
latest_sent[['bound_code', 'bank_code', 'receiver']].drop_duplicates()

,bound_code,bank_code,receiver
0,UPL_B5,vpbank,TP
93,UPL_B4,vpbank,TP
132,HQ_B5,vpbank,HQ
47412,VPB_01,vpbank,TP
48697,UPL_B3,vpbank,TP
51911,CC_V2,vpbank,TP
143877,CC_VNA,vpbank,HQ


In [17]:
latest_sent.groupby(['product_type', 'receiver']).\
    agg(count=('qualified', 'count'), qualified=('qualified', 'sum')).\
        reset_index()

,product_type,receiver,count,qualified
0,CC,HQ,317,317
1,CC,TP,46421,46421
2,UPL,HQ,4956,4956
3,UPL,TP,94240,94240


In [18]:
vp = latest_sent.loc[latest_sent['sent_at'] >= sys_date]
vp.head()

,lead_id,phone_number,national_id,bank_code,bound_code,sent_at,created_at,cs_version,sender,location,score,score_range,telco_code,qualified,fwd,recycled,receiver,product_type
0,14214455,84971775961,None,vpbank,UPL_B5,2023-06-30,2023-06-30,V4,sms,TP HCM,685,685-689,viettel,1,1,0,TP,UPL
127,14214767,84964144887,351938769,vpbank,UPL_B5,2023-06-30,2023-06-30,V4,sms,TP HCM,715,715-719,viettel,1,1,0,TP,UPL
118,14215226,84869151551,None,vpbank,UPL_B5,2023-06-30,2023-06-30,V4,sms,Bình Dương,700,700-704,viettel,1,1,0,TP,UPL
119,14215333,841646240339,None,vpbank,UPL_B5,2023-06-30,2023-06-30,V4,sms,TP HCM,730,730-734,viettel,1,1,0,TP,UPL
120,14215553,84869812595,None,vpbank,UPL_B5,2023-06-30,2023-06-30,V4,sms,TP HCM,790,790-794,viettel,1,1,0,TP,UPL


In [19]:
vp['phone_number'] = convert_to_new_prefix(vp['phone_number'])
vp.head()

/tmp/ipykernel_163461/3310166524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vp['phone_number'] = convert_to_new_prefix(vp['phone_number'])


,lead_id,phone_number,national_id,bank_code,bound_code,sent_at,created_at,cs_version,sender,location,score,score_range,telco_code,qualified,fwd,recycled,receiver,product_type
0,14214455,84971775961,None,vpbank,UPL_B5,2023-06-30,2023-06-30,V4,sms,TP HCM,685,685-689,viettel,1,1,0,TP,UPL
127,14214767,84964144887,351938769,vpbank,UPL_B5,2023-06-30,2023-06-30,V4,sms,TP HCM,715,715-719,viettel,1,1,0,TP,UPL
118,14215226,84869151551,None,vpbank,UPL_B5,2023-06-30,2023-06-30,V4,sms,Bình Dương,700,700-704,viettel,1,1,0,TP,UPL
119,14215333,84346240339,None,vpbank,UPL_B5,2023-06-30,2023-06-30,V4,sms,TP HCM,730,730-734,viettel,1,1,0,TP,UPL
120,14215553,84869812595,None,vpbank,UPL_B5,2023-06-30,2023-06-30,V4,sms,TP HCM,790,790-794,viettel,1,1,0,TP,UPL


3. JOIN TS & LOTTE DATA

In [20]:
vp_clean = vp
df_map_phone = pd.merge(df_all, vp_clean.drop("national_id", axis=1), on="phone_number")
df_map_phone[df_map_phone.duplicated(subset=["phone_number", "product_type_x"])]

,contract_no,product_type_x,product_detail,receiver_x,phone_number,customer_name,score_band,disbursed_date,loan_amount,fee_no_vat,fee_w_vat,national_id,lead_id,bank_code,bound_code,sent_at,created_at,cs_version,sender,location,score,score_range,telco_code,qualified,fwd,recycled,receiver_y,product_type_y


In [21]:
df_filtered = df_all[~df_all.phone_number.isin(vp.phone_number)]

df_map_nid = pd.merge(df_filtered, vp_clean.drop("phone_number", axis=1), on="national_id", how="inner")
df_map_nid

,contract_no,product_type_x,product_detail,receiver_x,phone_number,customer_name,score_band,disbursed_date,loan_amount,fee_no_vat,fee_w_vat,national_id,lead_id,bank_code,bound_code,sent_at,created_at,cs_version,sender,location,score,score_range,telco_code,qualified,fwd,recycled,receiver_y,product_type_y
0,LN2305299294756,UPL,10. UPL,HQ,84865161197,TRUONG QUANG THIEN,B05,2023-06-01,50000000,1.636364e+06,1800000.0,027097000193,14110887,vpbank,UPL_B5,2023-05-27,2023-05-27,V4,sms,Bắc Ninh,765,765-769,viettel,1,1,0,TP,UPL
1,LN2306099397543,UPL,10. UPL,HQ,84982275309,NGUYEN VAN HUNG,B05,2023-06-09,50000000,1.636364e+06,1800000.0,001082053589,14130734,vpbank,UPL_B5,2023-06-03,2023-06-03,V4,sms,Vĩnh Phúc,720,720-724,viettel,1,1,0,TP,UPL
2,LN2306129412620,UPL,10. UPL,HQ,84917328750,PHAM THUY LIEU,B05,2023-06-13,50000000,1.636364e+06,1800000.0,034177000506,14144760,vpbank,UPL_B5,2023-06-07,2023-06-07,V4,sms,Hà Nội,790,790-794,viettel,1,1,0,TP,UPL
3,LN2306159453267,UPL,10. UPL,HQ,84908398049,DUONG VAN TOAN,B05,2023-06-16,25000000,1.636364e+06,1800000.0,089089003878,14167255,vpbank,UPL_B5,2023-06-14,2023-06-14,V4,sms,Bình Dương,715,715-719,viettel,1,1,0,TP,UPL
4,LN2306019331930,UPL,10. UPL,HQ,84966389906,NGUYEN TRONG HIEU,B05,2023-06-05,80000000,1.818182e+06,2000000.0,001074001305,14046546,vpbank,UPL_B5,2023-05-08,2023-05-08,V4,sms,Hà Nội,705,705-709,viettel,1,1,0,TP,UPL
5,LN2306219502183,UPL,10. UPL,HQ,84833023400,DAO DUY NAM,B05,2023-06-21,35000000,1.636364e+06,1800000.0,001090006653,14174765,vpbank,UPL_B5,2023-06-17,2023-06-17,V4,sms,Hà Nội,715,715-719,viettel,1,1,0,TP,UPL
6,LN2306019334074,UPL,10. UPL,HQ,84335121263,TRAN DUC HUAN,B05,2023-06-02,44000000,1.636364e+06,1800000.0,001092013466,14112046,vpbank,UPL_B5,2023-05-27,2023-05-27,V4,sms,Hà Nội,730,730-734,viettel,1,1,0,TP,UPL
7,LN2305109123588,UPL,10. UPL,HQ,84966992768,TRAN CONG HOANG,B05,2023-06-01,80000000,1.818182e+06,2000000.0,044092000850,14036537,vpbank,UPL_B5,2023-05-05,2023-05-05,V4,sms,Bình Dương,705,705-709,viettel,1,1,0,TP,UPL
8,LN2306099400010,UPL,10. UPL,HQ,84919790866,BUI VAN BINH,B05,2023-06-09,46700000,1.636364e+06,1800000.0,001082010623,14145476,vpbank,UPL_B5,2023-06-08,2023-06-08,V4,sms,Hà Nội,715,715-719,viettel,1,1,0,TP,UPL
9,LN2306289573189,UPL,10. UPL,HQ,84936465681,NGO THI KHANH VAN,B05,2023-06-28,31200000,1.636364e+06,1800000.0,001179005643,14201824,vpbank,UPL_B5,2023-06-26,2023-06-26,V4,sms,Hà Nội,725,725-729,viettel,1,1,0,TP,UPL


In [22]:
df_map = pd.concat([df_map_phone, df_map_nid])
df_map

,contract_no,product_type_x,product_detail,receiver_x,phone_number,customer_name,score_band,disbursed_date,loan_amount,fee_no_vat,fee_w_vat,national_id,lead_id,bank_code,bound_code,sent_at,created_at,cs_version,sender,location,score,score_range,telco_code,qualified,fwd,recycled,receiver_y,product_type_y
0,LN2306089394251,UPL,10. UPL,HQ,84927057879,DAO THI TUYET,B05,2023-06-09,80000000,1.818182e+06,2000000.0,038172013234,14144455,vpbank,UPL_B5,2023-06-07,2023-06-07,V4,sms,Bình Dương,710,710-714,viettel,1,1,0,TP,UPL
1,LN2305309298673,UPL,10. UPL,HQ,84334181905,TRINH THU HA,B05,2023-06-01,40000000,1.636364e+06,1800000.0,001196000396,14105793,vpbank,UPL_B5,2023-05-25,2023-05-25,V4,sms,Hà Nội,715,715-719,viettel,1,1,0,TP,UPL
2,LN2306019333133,UPL,10. UPL,HQ,84398394845,LE VU HUONG LY,B05,2023-06-03,55000000,1.636364e+06,1800000.0,011199005604,14092789,vpbank,UPL_B5,2023-05-20,2023-05-20,V4,sms,Hà Nội,720,720-724,viettel,1,1,0,TP,UPL
3,LN2306099397341,UPL,10. UPL,HQ,84342904125,BUI ANH VU,B05,2023-06-13,70000000,1.818182e+06,2000000.0,064098007497,14145326,vpbank,UPL_B5,2023-06-08,2023-06-08,V4,sms,TP HCM,790,790-794,viettel,1,1,0,TP,UPL
4,LN2306069366736,UPL,10. UPL,HQ,84975288432,NGUYEN THI HANG NGA,B05,2023-06-07,30000000,1.636364e+06,1800000.0,001179021606,14127886,vpbank,UPL_B5,2023-06-02,2023-06-02,V4,sms,Hà Nội,690,690-694,viettel,1,1,0,TP,UPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,LN2306099399228,UPL,10. UPL,HQ,84931292878,DOAN HONG YEN,B05,2023-06-09,80000000,1.818182e+06,2000000.0,080177013781,14144981,vpbank,UPL_B5,2023-06-08,2023-06-08,V4,sms,Long An,715,715-719,viettel,1,1,0,TP,UPL
11,LN2306299585153,UPL,10. UPL,HQ,84906993197,HO XUAN SANG,B05,2023-06-29,60000000,1.636364e+06,1800000.0,070090010250,14204059,vpbank,UPL_B5,2023-06-27,2023-06-27,V4,sms,TP HCM,705,705-709,viettel,1,1,0,TP,UPL
12,LN2306169463837,UPL,10. UPL,HQ,84979331883,LE BANG QUANG,B05,2023-06-19,20000000,1.636364e+06,1800000.0,038081008736,14166386,vpbank,UPL_B5,2023-06-14,2023-06-14,V4,sms,Hà Nội,775,775-779,viettel,1,1,0,TP,UPL
13,LN2306219504865,UPL,10. UPL,HQ,84329582654,LA THI HAI YEN,B05,2023-06-21,80000000,1.818182e+06,2000000.0,034197015236,14180332,vpbank,UPL_B5,2023-06-19,2023-06-19,V4,sms,Bà Rịa - Vũng Tàu,725,725-729,viettel,1,1,0,TP,UPL


In [23]:
df_anti = pd.merge(df_all, df_map, on="contract_no", how="outer", indicator=True)
df_anti = df_anti[(df_anti['_merge'] == "left_only") | (df_anti['_merge'] == "right_only")].drop("_merge", axis=1)

df_anti = df_anti.dropna(axis=1, how='all')
df_anti.columns = df_anti.columns.str.rstrip('_x')
df_anti.head()

,contract_no,product_type,product_detail,receiver,phone_number,customer_name,score_band,disbursed_date,loan_amount,fee_no_vat,fee_w_vat,national_id
14,LN2306279562092,UPL,10. UPL,HQ,84919568169,VO VAN LAI,B05,2023-06-28,60000000,1.636364e+06,1800000.0,075068001711
64,LN2305309299651,UPL,10. UPL,HQ,84939126968,NGUYEN DUC TINH,B05,2023-06-01,50000000,1.636364e+06,1800000.0,037089011024
81,LN2306029339456,UPL,10. UPL,HQ,84918566166,NGUYEN VAN QUYNH,B05,2023-06-02,80000000,1.818182e+06,2000000.0,030091001110
112,LN2306219506114,UPL,10. UPL,HQ,84984226308,NGUYEN VAN THANG,B05,2023-06-22,50000000,1.636364e+06,1800000.0,001084017531
258,LN2306269550346,UPL,10. UPL,HQ,84947570992,LOAN HOANG PHONG,B05,2023-06-27,46100000,1.636364e+06,1800000.0,075081018023


In [112]:
df_map_cont = df_anti.merge(vp.drop('national_id', axis=1), on='phone_number', how='left')
df_map_all = pd.concat([df_map_phone, df_map_nid, df_map_cont])
df_map_all['lead_id'] = df_map_all['lead_id'].astype(str)
df_map_all

,contract_no,product_type_x,product_detail,receiver_x,phone_number,customer_name,score_band,disbursed_date,loan_amount,fee_no_vat,fee_w_vat,national_id,lead_id,bank_code,bound_code,sent_at,created_at,cs_version,sender,location,score,score_range,telco_code,qualified,fwd,recycled,receiver_y,product_type_y
0,LN2306089394251,UPL,10. UPL,HQ,84927057879,DAO THI TUYET,B05,2023-06-09,80000000,1.818182e+06,2000000.0,038172013234,14144455,vpbank,UPL_B5,2023-06-07,2023-06-07,V4,sms,Bình Dương,710.0,710-714,viettel,1.0,1.0,0.0,TP,UPL
1,LN2305309298673,UPL,10. UPL,HQ,84334181905,TRINH THU HA,B05,2023-06-01,40000000,1.636364e+06,1800000.0,001196000396,14105793,vpbank,UPL_B5,2023-05-25,2023-05-25,V4,sms,Hà Nội,715.0,715-719,viettel,1.0,1.0,0.0,TP,UPL
2,LN2306019333133,UPL,10. UPL,HQ,84398394845,LE VU HUONG LY,B05,2023-06-03,55000000,1.636364e+06,1800000.0,011199005604,14092789,vpbank,UPL_B5,2023-05-20,2023-05-20,V4,sms,Hà Nội,720.0,720-724,viettel,1.0,1.0,0.0,TP,UPL
3,LN2306099397341,UPL,10. UPL,HQ,84342904125,BUI ANH VU,B05,2023-06-13,70000000,1.818182e+06,2000000.0,064098007497,14145326,vpbank,UPL_B5,2023-06-08,2023-06-08,V4,sms,TP HCM,790.0,790-794,viettel,1.0,1.0,0.0,TP,UPL
4,LN2306069366736,UPL,10. UPL,HQ,84975288432,NGUYEN THI HANG NGA,B05,2023-06-07,30000000,1.636364e+06,1800000.0,001179021606,14127886,vpbank,UPL_B5,2023-06-02,2023-06-02,V4,sms,Hà Nội,690.0,690-694,viettel,1.0,1.0,0.0,TP,UPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,LN2306089391524,UPL,10. UPL,HQ,84978566682,NGUYEN VAN PHONG,B05,2023-06-09,65000000,1.636364e+06,1800000.0,024082006043,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,LN2306239532338,UPL,10. UPL,HQ,84764899520,THACH RACH TA NA,B05,2023-06-24,30000000,1.636364e+06,1800000.0,084096002125,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LN2306019334168,UPL,10. UPL,HQ,84913222962,NGUYEN VAN PHAN,B05,2023-06-15,80000000,1.818182e+06,2000000.0,027084002034,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LN2306209496421,UPL,10. UPL,HQ,84822938297,VU THI THANH NGAN,B05,2023-06-23,60000000,1.636364e+06,1800000.0,033199002579,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
df_map_all.drop_duplicates(subset=['contract_no', 'phone_number', 'product_type_x'])

print(df_map_all.shape)

df_map_all[df_map_all.duplicated(subset=['phone_number', 'product_type_x'], keep=False)].sort_values(by=['phone_number', 'product_type_x'])


(449, 28)


,contract_no,product_type_x,product_detail,receiver_x,phone_number,customer_name,score_band,disbursed_date,loan_amount,fee_no_vat,fee_w_vat,national_id,lead_id,bank_code,bound_code,sent_at,created_at,cs_version,sender,location,score,score_range,telco_code,qualified,fwd,recycled,receiver_y,product_type_y


In [114]:
df_map_all['sender'] = np.where(df_map_all['sender'].isna(), 'sms', df_map_all['sender'])
df_map_all

,contract_no,product_type_x,product_detail,receiver_x,phone_number,customer_name,score_band,disbursed_date,loan_amount,fee_no_vat,fee_w_vat,national_id,lead_id,bank_code,bound_code,sent_at,created_at,cs_version,sender,location,score,score_range,telco_code,qualified,fwd,recycled,receiver_y,product_type_y
0,LN2306089394251,UPL,10. UPL,HQ,84927057879,DAO THI TUYET,B05,2023-06-09,80000000,1.818182e+06,2000000.0,038172013234,14144455,vpbank,UPL_B5,2023-06-07,2023-06-07,V4,sms,Bình Dương,710.0,710-714,viettel,1.0,1.0,0.0,TP,UPL
1,LN2305309298673,UPL,10. UPL,HQ,84334181905,TRINH THU HA,B05,2023-06-01,40000000,1.636364e+06,1800000.0,001196000396,14105793,vpbank,UPL_B5,2023-05-25,2023-05-25,V4,sms,Hà Nội,715.0,715-719,viettel,1.0,1.0,0.0,TP,UPL
2,LN2306019333133,UPL,10. UPL,HQ,84398394845,LE VU HUONG LY,B05,2023-06-03,55000000,1.636364e+06,1800000.0,011199005604,14092789,vpbank,UPL_B5,2023-05-20,2023-05-20,V4,sms,Hà Nội,720.0,720-724,viettel,1.0,1.0,0.0,TP,UPL
3,LN2306099397341,UPL,10. UPL,HQ,84342904125,BUI ANH VU,B05,2023-06-13,70000000,1.818182e+06,2000000.0,064098007497,14145326,vpbank,UPL_B5,2023-06-08,2023-06-08,V4,sms,TP HCM,790.0,790-794,viettel,1.0,1.0,0.0,TP,UPL
4,LN2306069366736,UPL,10. UPL,HQ,84975288432,NGUYEN THI HANG NGA,B05,2023-06-07,30000000,1.636364e+06,1800000.0,001179021606,14127886,vpbank,UPL_B5,2023-06-02,2023-06-02,V4,sms,Hà Nội,690.0,690-694,viettel,1.0,1.0,0.0,TP,UPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,LN2306089391524,UPL,10. UPL,HQ,84978566682,NGUYEN VAN PHONG,B05,2023-06-09,65000000,1.636364e+06,1800000.0,024082006043,nan,NaN,NaN,NaN,NaN,NaN,sms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,LN2306239532338,UPL,10. UPL,HQ,84764899520,THACH RACH TA NA,B05,2023-06-24,30000000,1.636364e+06,1800000.0,084096002125,nan,NaN,NaN,NaN,NaN,NaN,sms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,LN2306019334168,UPL,10. UPL,HQ,84913222962,NGUYEN VAN PHAN,B05,2023-06-15,80000000,1.818182e+06,2000000.0,027084002034,nan,NaN,NaN,NaN,NaN,NaN,sms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,LN2306209496421,UPL,10. UPL,HQ,84822938297,VU THI THANH NGAN,B05,2023-06-23,60000000,1.636364e+06,1800000.0,033199002579,nan,NaN,NaN,NaN,NaN,NaN,sms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
unmap = df_map_all[df_map_all['lead_id'].isna()]
unmap

,contract_no,product_type_x,product_detail,receiver_x,phone_number,customer_name,score_band,disbursed_date,loan_amount,fee_no_vat,fee_w_vat,national_id,lead_id,bank_code,bound_code,sent_at,created_at,cs_version,sender,location,score,score_range,telco_code,qualified,fwd,recycled,receiver_y,product_type_y


In [116]:
print(f'Rate of not TS lead is: {unmap.shape[0] / df_map_all.shape[0] * 100}% (if smaller than 5% then no need further investigate)')

Rate of not TS lead is: 0.0% (if smaller than 5% then no need further investigate)


In [29]:
#pricing

# CC: VAT included:
# MC2: 495.000/card
# Titanimum: 495.000/card
# Platinum: 495.000/card

# Loan: included VAT
# B3: 1.000.000 VND
# B4: 1.400.000 VND with loan <= 43 tr and 1.600.000 if loan > 43 tr
# B5: 1.800.000 VND with loan <= 65 tr and 2.000.000 if loan > 65 tr

In [30]:
# check score band: 
# note: 
# ====band|---current V4---|-----------V3--------|---V2----|---old------|
#     B3  |  575-614       |  521-580 (521-579)  | 565-609 | 575- 609   |
#     B4  |  615-684       |  581-650 (580-650)  | 610-666 | 610-644    |
#     B5  |  >685          |  >650 (>=650)       | 667+    | 645+       |
#     CC  |  >626          |  584+               | 613+    | 575- 609   |
#     CC  |                |  630-689            | 613+    | 575- 609   |

def check_score_band_v2(df, score_band, cs_version):
    if score_band == 'B03':
        result = df[(df['product_type_x'] == 'UPL') & (df['score_band'] == score_band) & ((df['score'] < 575) | (df['score'] >= 610)) & (df['cs_version'] == cs_version)]
    elif score_band == 'B04':
        result = df[(df['product_type_x'] == 'UPL') & (df['score_band'] == score_band) & ((df['score'] < 610) | (df['score'] >= 645)) & (df['cs_version'] == cs_version)]
    elif score_band == 'B05':
        result = df[(df['product_type_x'] == 'UPL') & (df['score_band'] == score_band) & (df['score'] < 645) & (df['cs_version'] == cs_version)]
    elif score_band == 'CC':
        result = df[(df['product_type_x'] == score_band) & (df['score'] < 613) & (df['cs_version'] == cs_version)]
    else:
        result = df[(df['product_type_x'] == score_band) & (df['score'] < 630) & (df['cs_version'] == cs_version)]

    return result

In [31]:
print(check_score_band_v2(df_map_all, 'B03', 'V2'))
print(check_score_band_v2(df_map_all, 'B04', 'V2'))
print(check_score_band_v2(df_map_all, 'B05', 'V2'))
print(check_score_band_v2(df_map_all, 'CC', 'V2'))

Empty DataFrame
Columns: [contract_no, product_type_x, product_detail, receiver_x, phone_number, customer_name, score_band, disbursed_date, loan_amount, fee_no_vat, fee_w_vat, national_id, lead_id, bank_code, bound_code, sent_at, created_at, cs_version, sender, location, score, score_range, telco_code, qualified, fwd, recycled, receiver_y, product_type_y]
Index: []
Empty DataFrame
Columns: [contract_no, product_type_x, product_detail, receiver_x, phone_number, customer_name, score_band, disbursed_date, loan_amount, fee_no_vat, fee_w_vat, national_id, lead_id, bank_code, bound_code, sent_at, created_at, cs_version, sender, location, score, score_range, telco_code, qualified, fwd, recycled, receiver_y, product_type_y]
Index: []
Empty DataFrame
Columns: [contract_no, product_type_x, product_detail, receiver_x, phone_number, customer_name, score_band, disbursed_date, loan_amount, fee_no_vat, fee_w_vat, national_id, lead_id, bank_code, bound_code, sent_at, created_at, cs_version, sender, lo

In [32]:
def check_score_band_v3(df, score_band, cs_version):
    if score_band == 'B03':
        result = df[(df['product_type_x'] == 'UPL') & (df['score_band'] == score_band) & ((df['score'] < 521) | (df['score'] > 579)) & (df['cs_version'] == cs_version)]
    elif score_band == 'B04':
        result = df[(df['product_type_x'] == 'UPL') & (df['score_band'] == score_band) & ((df['score'] < 580) | (df['score'] > 650)) & (df['cs_version'] == cs_version)]
    elif score_band == 'B05':
        result = df[(df['product_type_x'] == 'UPL') & (df['score_band'] == score_band) & (df['score'] < 650) & (df['cs_version'] == cs_version)]
    elif score_band == 'CC':
        result = df[(df['product_type_x'] == score_band) & (df['score'] < 630) & (df['cs_version'] == cs_version)]
    else:
        result = df[(df['product_type_x'] == score_band) & (df['score'] < 630) & (df['cs_version'] == cs_version)]

    return result


In [33]:
print(check_score_band_v3(df_map_all, 'B03', 'V3'))
print(check_score_band_v3(df_map_all, 'B04', 'V3'))
print(check_score_band_v3(df_map_all, 'B05', 'V3'))
print(check_score_band_v3(df_map_all, 'CC', 'V3'))

Empty DataFrame
Columns: [contract_no, product_type_x, product_detail, receiver_x, phone_number, customer_name, score_band, disbursed_date, loan_amount, fee_no_vat, fee_w_vat, national_id, lead_id, bank_code, bound_code, sent_at, created_at, cs_version, sender, location, score, score_range, telco_code, qualified, fwd, recycled, receiver_y, product_type_y]
Index: []
Empty DataFrame
Columns: [contract_no, product_type_x, product_detail, receiver_x, phone_number, customer_name, score_band, disbursed_date, loan_amount, fee_no_vat, fee_w_vat, national_id, lead_id, bank_code, bound_code, sent_at, created_at, cs_version, sender, location, score, score_range, telco_code, qualified, fwd, recycled, receiver_y, product_type_y]
Index: []
Empty DataFrame
Columns: [contract_no, product_type_x, product_detail, receiver_x, phone_number, customer_name, score_band, disbursed_date, loan_amount, fee_no_vat, fee_w_vat, national_id, lead_id, bank_code, bound_code, sent_at, created_at, cs_version, sender, lo

In [34]:
def check_score_band_v4(df, score_band, cs_version):
    if score_band == 'B03':
        result = df[(df['product_type_x'] == 'UPL') & (df['score_band'] == score_band) & ((df['score'] < 575) | (df['score'] > 614)) & (df['cs_version'] == cs_version)]
    elif score_band == 'B04':
        result = df[(df['product_type_x'] == 'UPL') & (df['score_band'] == score_band) & ((df['score'] < 615) | (df['score'] > 684)) & (df['cs_version'] == cs_version)]
    elif score_band == 'B05':
        result = df[(df['product_type_x'] == 'UPL') & (df['score_band'] == score_band) & (df['score'] < 685) & (df['cs_version'] == cs_version)]
    elif score_band == 'CC':
        result = df[(df['product_type_x'] == score_band) & (df['score'] < 626) & (df['cs_version'] == cs_version)]
    else:
        result = df[(df['product_type_x'] == score_band) & (df['score'] < 630) & (df['cs_version'] == cs_version)]

    return result


In [35]:
print(check_score_band_v4(df_map_all, 'B03', 'V4'))
print(check_score_band_v4(df_map_all, 'B04', 'V4'))
print(check_score_band_v4(df_map_all, 'B05', 'V4'))
print(check_score_band_v4(df_map_all, 'CC', 'V4'))

Empty DataFrame
Columns: [contract_no, product_type_x, product_detail, receiver_x, phone_number, customer_name, score_band, disbursed_date, loan_amount, fee_no_vat, fee_w_vat, national_id, lead_id, bank_code, bound_code, sent_at, created_at, cs_version, sender, location, score, score_range, telco_code, qualified, fwd, recycled, receiver_y, product_type_y]
Index: []
Empty DataFrame
Columns: [contract_no, product_type_x, product_detail, receiver_x, phone_number, customer_name, score_band, disbursed_date, loan_amount, fee_no_vat, fee_w_vat, national_id, lead_id, bank_code, bound_code, sent_at, created_at, cs_version, sender, location, score, score_range, telco_code, qualified, fwd, recycled, receiver_y, product_type_y]
Index: []
Empty DataFrame
Columns: [contract_no, product_type_x, product_detail, receiver_x, phone_number, customer_name, score_band, disbursed_date, loan_amount, fee_no_vat, fee_w_vat, national_id, lead_id, bank_code, bound_code, sent_at, created_at, cs_version, sender, lo

In [117]:
df_map_all['loan_amount'] = pd.to_numeric(df_map_all['loan_amount'])

df_map_all['new_price_vat'] = np.where(
    (df_map_all['product_type_x'] == 'UPL') & (df_map_all['score_band'] == 'B03'),1000000,
    np.where(
        ((df_map_all['product_type_x'] == 'UPL') & (df_map_all['score_band'] == 'B04') & (df_map_all['loan_amount'] <= 43000000)),1400000,
        np.where(
            ((df_map_all['product_type_x'] == 'UPL') & (df_map_all['score_band'] == 'B04') & (df_map_all['loan_amount'] > 43000000)),
            1600000,
            np.where(
                ((df_map_all['product_type_x'] == 'UPL') & (df_map_all['score_band'] == 'B05') & (df_map_all['loan_amount'] <= 65000000)),
                1800000,
                np.where(
                    ((df_map_all['product_type_x'] == 'UPL') & (df_map_all['score_band'] == 'B05') & (df_map_all['loan_amount'] > 65000000)),
                    2000000,
                    np.where(
                        ((df_map_all['product_type_x'] == 'CC') & (df_map_all['product_detail'].str.contains('TITANIUM'))),
                        495000,
                        np.where(
                            ((df_map_all['product_type_x'] == 'CC') & (df_map_all['product_detail'].str.contains('PLATINUM'))),
                            495000,
                            np.where(
                                ((df_map_all['product_type_x'] == 'CC') & (~df_map_all['product_detail'].str.contains('PLATINUM')) & (~df_map_all['product_detail'].str.contains('TITANIUM'))),
                                495000,
                                0
                            )
                        )
                    )
                )
            )
        )
    )
)

In [118]:
df_map_all['new_price_vat'] = pd.to_numeric(df_map_all['new_price_vat'])
df_map_all['fee_w_vat'] = pd.to_numeric(df_map_all['fee_w_vat'])
df_map_all['lead_id'] = df_map_all['lead_id'].astype(str)
df_map_all[(df_map_all['product_type_x']=='UPL') & (df_map_all['new_price_vat'] != df_map_all['fee_w_vat'])]

,contract_no,product_type_x,product_detail,receiver_x,phone_number,customer_name,score_band,disbursed_date,loan_amount,fee_no_vat,fee_w_vat,national_id,lead_id,bank_code,bound_code,sent_at,created_at,cs_version,sender,location,score,score_range,telco_code,qualified,fwd,recycled,receiver_y,product_type_y,new_price_vat


In [119]:
df_map_all[df_map_all['product_type_x']=='CC'].groupby('product_detail').agg(
    {'product_detail':'count', 'fee_w_vat':'mean', 'new_price_vat':'mean'})

,product_detail,fee_w_vat,new_price_vat
product_detail,,,
14. VPBANK NO 1 CREDIT,9,495000.0,495000.0


In [189]:
summary_fee = df_map_all.groupby(['receiver_x', 'product_type_x']).agg({ 'lead_id':'count', 'new_price_vat':'sum'}).reset_index() 
summary_fee['new_price_exc_vat'] = summary_fee['new_price_vat']/1.1
total_row = summary_fee[['lead_id', 'new_price_vat', 'new_price_exc_vat']].sum() 
total_row['receiver_x'] = 'Sum'
total_row['product_type_x'] = ''
summary_fee = summary_fee.append(total_row, ignore_index=True)
create_or_update_sheet(wb = summary_fee_report, new_sheet = 'summary', input_df = summary_fee)
summary_fee.to_csv('summary_fee.csv', index = False)
summary_fee

/tmp/ipykernel_163461/1377578604.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_fee = summary_fee.append(total_row, ignore_index=True)


,receiver_x,product_type_x,lead_id,new_price_vat,new_price_exc_vat
0,HQ,CC,9.0,4455000.0,4.050000e+06
1,HQ,UPL,440.0,813800000.0,7.398182e+08
2,Sum,,449.0,818255000.0,7.438682e+08


In [190]:
summary_fee_all = df_map_all.groupby(['receiver_x']).agg({'lead_id':'count', 'new_price_vat':sum}).reset_index().rename(columns={'lead_id':'count','new_price_vat':'fee_w_vat'})

summary_fee_all['fee_exc_vat'] = summary_fee_all['fee_w_vat'] / 1.1

summary_fee_all['product_type_x'] = ''
summary_fee_all['receiver_x'] = 'Total'
summary_fee_all[['receiver_x', 'product_type_x', 'count', 'fee_w_vat', 'fee_exc_vat']]
summary_fee_all.to_csv('summary_fee_all.csv', index = False)
summary_fee_all

,receiver_x,count,fee_w_vat,fee_exc_vat,product_type_x
0,Total,449,818255000,7.438682e+08,


In [121]:
df_filtered = df_map_all[df_map_all['product_type_x']=='CC'].groupby('product_detail').agg({'product_detail':'count', 'new_price_vat':'mean'}).rename(columns={'product_detail':'count', 'new_price_vat':'avg'})
df_filtered

,count,avg
product_detail,,
14. VPBANK NO 1 CREDIT,9,495000.0


In [122]:
create_or_update_sheet(wb = vpb_converted_report, new_sheet = 'details', input_df = df_map_all)

In [123]:
df_summary = df_map_all.groupby(['product_type_x', 'receiver_x']).agg({'lead_id':'count', 'fee_no_vat':lambda x: sum(x)/1.1, 'fee_w_vat':'sum'}).rename(columns = {'lead_id':'number_converted'}).reset_index()
total_row = df_summary[['number_converted', 'fee_no_vat', 'fee_w_vat']].sum() 
total_row['product_type_x'] = 'Sum'
total_row['receiver_x'] = ''
df_summary = df_summary.append(total_row, ignore_index=True)

create_or_update_sheet(wb = vpb_converted_cycle_report, new_sheet = 'summary', input_df = df_summary)
df_summary

/tmp/ipykernel_163461/323960416.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_summary = df_summary.append(total_row, ignore_index=True)


,product_type_x,receiver_x,number_converted,fee_no_vat,fee_w_vat
0,CC,HQ,9.0,3.681818e+06,4455000.0
1,UPL,HQ,440.0,6.725620e+08,813800000.0
2,Sum,,449.0,6.762438e+08,818255000.0


In [176]:
df_upl = df_map_all[(df_map_all['product_type_x']=='UPL') & (df_map_all['receiver_x']=='HQ')].copy()
df_upl = df_upl[['contract_no', 'product_type_x', 'receiver_x', 'phone_number', 'customer_name',
                  'lead_id', 'score_band', 'disbursed_date', 'loan_amount', 'national_id', 'new_price_vat']].rename(
                      columns={'product_type_x':'product_type', 'receiver_x':'receiver'})
df_upl['new_price_exc_vat'] = df_upl['new_price_vat']/1.1
create_or_update_sheet(wb = vpb_converted_cycle_report, new_sheet = 'upl_hq', input_df = df_upl)
df_upl.to_csv('vpb_converted_cycle_report_upl.csv', index = False)

In [177]:
df_cc = df_map_all[(df_map_all['product_type_x']=='CC') & (df_map_all['receiver_x']=='HQ')].copy()
df_cc = df_cc[['contract_no', 'product_type_x', 'receiver_x', 'phone_number', 'customer_name', 'lead_id', 
               'score_band', 'disbursed_date', 'loan_amount', 'national_id', 'new_price_vat', 'product_detail']].rename(
                   columns={'product_type_x':'product_type', 'receiver.x':'receiver'})
df_cc['new_price_exc_vat'] = df_cc['new_price_vat']/1.1
create_or_update_sheet(wb = vpb_converted_cycle_report, new_sheet = 'cc_hq', input_df = df_cc)
df_cc.to_csv('vpb_converted_cycle_report_cc.csv', index = False)

In [126]:
df_disbursed = df_map_all.copy()
df_disbursed['fee_no_vat'] = df_disbursed['new_price_vat']/1.1
df_disbursed = df_disbursed[['bank_code', 'lead_id', 'phone_number', 'sent_at', 'sender', 'bound_code', 
                             'national_id', 'loan_amount', 'disbursed_date', 'fee_no_vat', 
                             'product_type_x', 'product_type_y', 'telco_code', 'score_band']].rename(
                                 columns={'product_type_x':'final_product', 'product_type_y':'offer_product'})
vp['lead_id'] = vp['lead_id'].astype(str)
df_disbursed['lead_id'] = df_disbursed['lead_id'].astype(str)
df_disbursed = pd.merge(df_disbursed, vp[['lead_id', 'national_id']], on='lead_id', how='left')
diff = df_disbursed[df_disbursed['national_id_x'] != df_disbursed['national_id_y']]
diff.head()

/tmp/ipykernel_163461/641239309.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vp['lead_id'] = vp['lead_id'].astype(str)


,bank_code,lead_id,phone_number,sent_at,sender,bound_code,national_id_x,loan_amount,disbursed_date,fee_no_vat,final_product,offer_product,telco_code,score_band,national_id_y
0,vpbank,14144455,84927057879,2023-06-07,sms,UPL_B5,038172013234,80000000.0,2023-06-09,1.818182e+06,UPL,UPL,viettel,B05,None
2,vpbank,14092789,84398394845,2023-05-20,sms,UPL_B5,011199005604,55000000.0,2023-06-03,1.636364e+06,UPL,UPL,viettel,B05,None
7,vpbank,14152429,84977501145,2023-06-10,sms,UPL_B5,030173012561,30000000.0,2023-06-14,1.636364e+06,UPL,UPL,viettel,B05,024950509
12,vpbank,14193307,84972995714,2023-06-23,sms,UPL_B5,077092004186,80000000.0,2023-06-26,1.818182e+06,UPL,UPL,viettel,B05,None
14,vpbank,14172326,84357815212,2023-06-16,sms,UPL_B5,017183004664,80000000.0,2023-06-19,1.818182e+06,UPL,UPL,viettel,B05,None


In [127]:
df_disbursed['type_of_sale'] = np.select(
        [
            df_disbursed['offer_product'].isna(),
            df_disbursed['final_product'] == df_disbursed['offer_product']
        ],
        [
            'lead_not_found',
            'direct_sale'
        ],
        default='cross_sale'
    )
df_disbursed['bank_code'] = np.where(df_disbursed['bank_code'].notna(), df_disbursed['bank_code'], 'vpbank')
df_disbursed['bound_code'] = np.where(df_disbursed['type_of_sale'] != 'lead_not_found', df_disbursed['bound_code'], df_disbursed['score_band'])

df_disbursed = df_disbursed[['bank_code', 'lead_id', 'phone_number', 'sent_at', 'sender', 'bound_code', 'national_id_y', 'telco_code', 'loan_amount', 'disbursed_date', 'fee_no_vat', 'type_of_sale', 'final_product']].rename(columns={'national_id_y':'nid'})

df_disbursed = df_disbursed.assign(
    tenor=np.nan,
    fee_rate=df_disbursed['fee_no_vat']/df_disbursed['loan_amount'],
    disbursed_status='reconciled'
)
df_disbursed = df_disbursed[['bank_code', 'lead_id', 'phone_number', 'sent_at', 'sender', 'bound_code', 
                             'nid', 'telco_code', 'tenor', 'loan_amount', 'disbursed_date', 'fee_rate', 
                             'fee_no_vat', 'disbursed_status', 'type_of_sale']].rename(columns={
                                 'sent_at':'batch_date', 'sender':'channel', 'bound_code':'product_code', 
                                 'type_of_sale':'other', 'fee_no_vat':'commission'})
df_disbursed = df_disbursed[df_disbursed['bank_code'].notna()]
df_disbursed[df_disbursed['other']=='lead_not_found']
df_disbursed.groupby('other').agg({'lead_id':'count'})

,lead_id
other,
cross_sale,9
direct_sale,430
lead_not_found,10


In [128]:
df_disbursed.groupby('channel').agg({'lead_id':'count'})

,lead_id
channel,
avay,7
sms,442


In [99]:
#add lead source
avay_bq_acc = AvayBQServiceAcc()

avay_df = avay_bq_acc.client.query(f"""
select case when source = 'prod.sendo' then 'Sendo'
        when source = 'prod.chotot' then 'ChoTot'
        when source = 'prod.kalapa' then 'Kalapa'
        when source = 'prod.vpp.viettel' then 'Viettel Pay Pro'
        when source = 'prod.vtp.viettel' then 'Viettel Pay'
        when utm_source like 'dir%' then utm_source
        when source = 'vaycucde.vn' then 'Vaycucde'
        when utm_source = 'fimar' then 'Fimar'
        when utm_source like '%ccesstra%' then 'Accesstrade'
        when utm_source = 'masoffer' then 'Masoffer'
        when utm_source = 'vaysieude' then 'Vaysieude'
        when utm_source = 'thomaytaichinh' then 'Thomaytaichinh'
        when utm_source = 'adpia' THEN 'Adpia'
        when utm_source like '%acebook%' or utm_source like '%fb%' or utm_source like 'chatfuel%' then 'Facebook'
        when source = 'android_app' then 'Android App'
        when utm_campaign = 'adtima' or utm_source like 'zal%' then 'Zalo'
        when utm_source = 'sales_doubler' then 'Sale Doubler'
        when source = 'prod.zalo' then 'Zalo CC'
        when utm_source like 'google_ad%' then 'Google ads'
        when utm_source = 'zns' then 'ZNS'
        when utm_source = 'mpcc' then 'mpcc'
        when utm_source like 'mgi%' then 'MGID'
        when utm_source = 'fingo' then 'Fingo'
        when utm_source = 'esmart' then 'eSmart'
        when utm_source = 'linh_dan_spa' then 'Linh Dan Spa'
        when utm_source = 'cafe_anh' then 'Cafe Anh'
        when utm_source = 'tanvd' then 'Tanvd'
        when utm_source = 'adfly' then 'Adfly'
        when utm_source = 'coccoc' then 'Coccoc'
        when utm_source in ('adskeeper','adskeeper.com') then 'Adskeeper'
        when utm_source = 'vinfin' then 'Vinfin'
        when utm_source = 'propellerads' then 'Propellerads'
        when utm_source = 'dinos' then 'Dinos'
        when utm_source = 'clickadu' then 'Clickadu'
        else 'Others' end as lead_source
    ,lead_id
  from `avay-a9925.dwh.loans` l
  left join `avay-a9925.dwh.registrations` r on r.id = l.reg_id
  left join `avay-a9925.dwh.otps` o on r.otp_id = o.id
  where l.lead_id is not null
                          ;""").result().to_arrow().to_pandas()
  
avay_df

,lead_source,lead_id
0,Vaysieude,11112304
1,Dinos,13631658
2,Dinos,13352879
3,Adfly,9522116
4,Adfly,8763687
...,...,...
3650813,Viettel Pay,12104150
3650814,Viettel Pay,8568828
3650815,Viettel Pay,10527262
3650816,Viettel Pay,10574206


In [130]:
avay_df['lead_id'] = avay_df['lead_id'].astype(str)
df_disbursed['lead_id'] = df_disbursed['lead_id'].astype(str)
vpb_final_add_source = pd.merge(df_disbursed, avay_df, how='left', on='lead_id')
vpb_final_add_source

,bank_code,lead_id,phone_number,batch_date,channel,product_code,nid,telco_code,tenor,loan_amount,disbursed_date,fee_rate,commission,disbursed_status,other,lead_source
0,vpbank,14144455,84927057879,2023-06-07,sms,UPL_B5,None,viettel,NaN,80000000.0,2023-06-09,0.022727,1.818182e+06,reconciled,direct_sale,NaN
1,vpbank,14105793,84334181905,2023-05-25,sms,UPL_B5,001196000396,viettel,NaN,40000000.0,2023-06-01,0.040909,1.636364e+06,reconciled,direct_sale,NaN
2,vpbank,14092789,84398394845,2023-05-20,sms,UPL_B5,None,viettel,NaN,55000000.0,2023-06-03,0.029752,1.636364e+06,reconciled,direct_sale,NaN
3,vpbank,14145326,84342904125,2023-06-08,sms,UPL_B5,064098007497,viettel,NaN,70000000.0,2023-06-13,0.025974,1.818182e+06,reconciled,direct_sale,NaN
4,vpbank,14127886,84975288432,2023-06-02,sms,UPL_B5,001179021606,viettel,NaN,30000000.0,2023-06-07,0.054545,1.636364e+06,reconciled,direct_sale,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,vpbank,nan,84978566682,NaN,sms,B05,NaN,NaN,NaN,65000000.0,2023-06-09,0.025175,1.636364e+06,reconciled,lead_not_found,NaN
445,vpbank,nan,84764899520,NaN,sms,B05,NaN,NaN,NaN,30000000.0,2023-06-24,0.054545,1.636364e+06,reconciled,lead_not_found,NaN
446,vpbank,nan,84913222962,NaN,sms,B05,NaN,NaN,NaN,80000000.0,2023-06-15,0.022727,1.818182e+06,reconciled,lead_not_found,NaN
447,vpbank,nan,84822938297,NaN,sms,B05,NaN,NaN,NaN,60000000.0,2023-06-23,0.027273,1.636364e+06,reconciled,lead_not_found,NaN


In [131]:
vpb_final_add_source[(vpb_final_add_source['channel'] == 'avay') & (vpb_final_add_source['lead_source'].isna())]

,bank_code,lead_id,phone_number,batch_date,channel,product_code,nid,telco_code,tenor,loan_amount,disbursed_date,fee_rate,commission,disbursed_status,other,lead_source


In [133]:
vpb_final_add_source.loc[vpb_final_add_source['commission'] == 450000, 'loan_amount'] = 0
vpb_final_add_source.to_csv(f"{path}{month}/disbursals_vpb_{month}.csv", index=False)

In [135]:
file_path: Path = Path(f"{path}{month}/disbursals_vpb_{month}.csv")

# Send success message to channel
clientSlack.chat_postMessage(**{"text": f"<@{user_id}> update disbursals VPB {recon_month}"} , channel=channel_id)

# Upload file to channel
filetype = file_path.suffix[1:]
filename = file_path.name
with open(file_path, "rb") as file:
	clientSlack.files_upload(channels=channel_id, file=file, title=filename, filetype=filetype)

In [136]:
drive_repo.upload_file_to_drive(file_path= file_path, parent_folder_id= folder_month)


'1jifxHlRkyLO7XZwPP1U3pXVz24kKSBj-'

In [168]:
df_upl.drop(['product_type', 'receiver', 'national_id', 'phone_number', 'contract_no'], axis=1, inplace=True) 
df_upl.rename(columns={'new_price_vat':'fee_w_vat', 'new_price_exc_vat':'fee_no_vat'}, inplace=True)

In [169]:
df_upl['disbursed_date'] = df_upl['disbursed_date'].dt.date


In [170]:
df_upl

,customer_name,lead_id,score_band,disbursed_date,loan_amount,fee_w_vat,fee_no_vat
0,DAO THI TUYET,14144455,B05,2023-06-09,80000000.0,2000000,1.818182e+06
1,TRINH THU HA,14105793,B05,2023-06-01,40000000.0,1800000,1.636364e+06
2,LE VU HUONG LY,14092789,B05,2023-06-03,55000000.0,1800000,1.636364e+06
3,BUI ANH VU,14145326,B05,2023-06-13,70000000.0,2000000,1.818182e+06
4,NGUYEN THI HANG NGA,14127886,B05,2023-06-07,30000000.0,1800000,1.636364e+06
...,...,...,...,...,...,...,...
5,NGUYEN VAN PHONG,nan,B05,2023-06-09,65000000.0,1800000,1.636364e+06
6,THACH RACH TA NA,nan,B05,2023-06-24,30000000.0,1800000,1.636364e+06
7,NGUYEN VAN PHAN,nan,B05,2023-06-15,80000000.0,2000000,1.818182e+06
8,VU THI THANH NGAN,nan,B05,2023-06-23,60000000.0,1800000,1.636364e+06


In [171]:
create_or_update_sheet(wb = vpb_statement_report, new_sheet = 'upl_hq', input_df = df_upl)

In [173]:
df_cc.drop(['product_type', 'national_id', 'phone_number', 'contract_no', 'score_band', 'loan_amount'], axis=1, inplace=True)
df_cc['disbursed_date'] = df_cc['disbursed_date'].dt.date
df_cc = df_cc[['customer_name', 'product_detail', 'disbursed_date', 'new_price_exc_vat', 'new_price_vat']].rename(columns={'new_price_exc_vat':'fee_no_vat', 'new_price_vat':'fee_with_vat'})

In [174]:
create_or_update_sheet(wb = vpb_statement_report, new_sheet = 'cc_hq', input_df = df_cc)